### Data preprocessing components: 

* DocumentCleaner 

* DocumentSplitter 

### Data extraction components: 

* LinkContentFetcher 

### Data caching components: 

* URLCacheChecker 

### Audio to text processing components: 

* LocalWhisperTranscriber 

* RemoteWhisperTranscriber 

### File converter components: 

* AzureOCRDocumentConverter 

* HTMLToDocument 

* MarkdownToDocument 

* PyPDFToDocument 

* TikaDocumentConverter 

* TextFileToDocument 

### Language classifier components: 

* DocumentLanguageClassifier 

* TextLanguageClassifier 

### Document Cleaner

In [4]:
from haystack.preview.components.preprocessors import DocumentCleaner 
from haystack.preview.dataclasses import Document

Simple instance removing extra white spaces, specific characters. We can also remove special characters.

In [21]:
# Define a regular expression for removing exclamation marks and punctuation
punctuation_regex = r"[!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~]"

# Create an instance of DocumentCleaner with the regex
cleaner = DocumentCleaner(
    remove_empty_lines=True,
    remove_extra_whitespaces=True,
    remove_repeated_substrings=False,
    remove_substrings=punctuation_regex,
    remove_regex=None
)

# Sample document with exclamation marks and punctuation
sample_document = Document(content="This is a simple document! <<With some extra spaces... and punctuation!!", meta={"name": "test_doc"})

# Using the cleaner
cleaned_documents = cleaner.run([sample_document])

# Extracting the cleaned document
cleaned_document = cleaned_documents['documents'][0]

# Output the cleaned content
print("Cleaned Document Content:", cleaned_document.content)

Cleaned Document Content: This is a simple document With some extra spaces and punctuation
